# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [4]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = ''  # Please insert by own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [5]:
# Define some util functions
def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [6]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'guitar',
        'filter': 'ac_warmth:[40 TO 100] AND duration:[0 TO 10]',
        'num_results': 20, 
    },
    {
        'query': 'bird',
        'filter': 'ac_brightness:[30 TO 100] AND duration:[0 TO 10]',
        'num_results': 20, 
    },
    {
        'query': 'book',
        'filter': 'duration:[0 TO 10]',
        'num_results': 10, 
    },
    {
        'query': 'wave sea',
        'filter': 'duration:[0 TO 5]',
        'num_results': 5, 
    },
    {
        'query': 'washing machine',
        'filter': 'duration:[0 TO 5]',
        'num_results': 10, 
    },
    {
        'query': 'car horns',
        'filter': 'ac_reverb:True AND duration:[0 TO 10]',
        'num_results': 7, 
    },
    {
        'query': 'mobile phone ring',
        'filter': 'duration:[0 TO 20]',
        'num_results': 10, 
    },
    {
        'query': 'scream',
        'filter': 'ac_reverb:True',
        'num_results': 7, 
    }, 
    {
        'query': 'door slam',
        'filter': 'duration:[0 TO 4]',
        'num_results': 10, 
    },
    {
        'query': 'camera flash',
        'filter': 'duration:[0 TO 10]',
        'num_results': 10, 
    },
    {
        'query': 'dog bark',
        'filter': 'duration:[0 TO 5]',
        'num_results': 5, 
    }
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 114 entries! dataframe.csv


,freesound_id,license,name,path,tags,username
0,362156,http://creativecommons.org/licenses/by/3.0/,"Guitar Pitch Pipe, C3.wav",files/362156_5121236-hq.ogg,"[pitch, 3, Instrument, Guitar, Pitch, sampler,...",InspectorJ
1,181425,http://creativecommons.org/publicdomain/zero/1.0/,Guitar chord.wav,files/181425_1823830-hq.ogg,"[a, guitar, microphone, from, chords, game, Gu...",SeryLis
2,439596,http://creativecommons.org/licenses/by/3.0/,"Guitar, Acoustic, Slide Down, Ab.wav",files/439596_5121236-hq.ogg,"[glissandi, down, Sliding, acoustic, Acoustic,...",InspectorJ
3,203647,http://creativecommons.org/publicdomain/zero/1.0/,Guitar-Chord Ebm+Reverb+E-Guitar,files/203647_3661618-hq.ogg,"[Guitar, Ebm, Guitar-Chord]",Jack_Master
4,16054,http://creativecommons.org/licenses/by/3.0/,Guitar clean rif.wav,files/16054_36188-hq.ogg,"[clean, guitar, intro, riff]",LG
5,40404,http://creativecommons.org/licenses/by/3.0/,Guitare3 02.wav,files/40404_417293-hq.ogg,"[electric, guitar, loop]",hubertmichel
6,1377,http://creativecommons.org/licenses/sampling+/...,raked_guitar.wav,files/1377_838-hq.ogg,"[guitar, nylon, rasgueado]",sleep
7,186498,http://creativecommons.org/licenses/by-nc/3.0/,guitar2.aif,files/186498_2250420-hq.ogg,"[guitar, acoustic, oragnic]",RistoMilner
8,121776,http://creativecommons.org/publicdomain/zero/1.0/,guitar2.wav,files/121776_907272-hq.ogg,"[ambient, delay, guitar, guitarloop, iceland, ...",Pax11
9,58900,http://creativecommons.org/licenses/by/3.0/,ST Slide guitar blues riff 3.wav,files/58900_649468-hq.ogg,"[acoustic, blues, guitar, oetter, riff, slide]",juskiddink
